In [15]:
import pandas as pd
import re
import os
import regex as re
import string
import csv
import jieba

## 1. Extract the review data from csv and save it as txt

In [16]:
# Define the file paths to read from
file_paths = ['../data/北京apm.csv', '../data/王府中環.csv', '../data/王府井百货.csv', '../data/东方新天地.csv']

# Loop over each file, extract the data from the 'review' column and save it to a text file
for i, path in enumerate(file_paths):
    # Read the CSV file
    df = pd.read_csv(path, index_col=0)  # Set the index_col to 0 to use the first column as the index
    # Define the path for the output text file
    txt_path = f'../data/review/reviews_{i+1}.txt'
    # Write the review data to the output text file
    with open(txt_path, 'w') as f:
        for j, review in enumerate(df['review']):
            # Clean up the text data by removing special characters
            review = re.sub('[\n\r\t ]+', '', review)
            # Write the review data to the output text file
            f.write(str(review) + '\n')

## 2. Formatting (Remove emoji, spaces & Replace punctuations)

In [17]:
# Define the file paths to modify
txt_paths = ['../data/review/reviews_1.txt', '../data/review/reviews_2.txt', '../data/review/reviews_3.txt', '../data/review/reviews_4.txt']

# Define a string of all punctuation marks
all_punct = string.punctuation + '，。？！：；（）“”‘’【】『』《》〈〉·—…～、｜「」'

# Define regular expression matching pattern
pattern = re.compile(',+')

# Loop over each file and modify its contents
for txt_path in txt_paths:
    # Open the file for reading
    with open(txt_path, 'r') as file:
        # Read the contents of the file and replace all punctuation marks with commas
        contents = file.read().translate(str.maketrans(all_punct, ',' * len(all_punct)))
        
        # Remove all emoji symbols
        contents = re.sub(r'\p{Emoji}', '', contents)
        
        # Replacing consecutive commas
        contents = re.sub(pattern, ',', contents)
        
        # Remove trailing commas from each line
        contents = '\n'.join(line.rstrip(',') for line in contents.split('\n'))
        
    # Open the file for writing
    with open(txt_path, 'w') as file:
        # Write the modified contents to the file
        file.write(contents)


## 3. Merge into a new txt

In [18]:
# Define the file paths to read from
txt_paths = ['../data/review/reviews_1.txt', '../data/review/reviews_2.txt', '../data/review/reviews_3.txt', '../data/review/reviews_4.txt']

# Define the path for the output merged text file
merged_txt_path = '../data/review/reviews_merged.txt'

# Open the output file for writing
with open(merged_txt_path, 'w', newline='\n') as merged_file:
    # Loop over each input file
    for txt_path in txt_paths:
        # Open the input file for reading
        with open(txt_path, 'r', newline='') as input_file:
            # Loop over each line in the input file
            for line in input_file:
                # Write the line to the output file
                merged_file.write(line.rstrip('\r\n') + '\n')


### Read into dataframe to validate

In [19]:
# Define the path to the merged text file
merged_txt_path = '../data/review/reviews_merged.txt'

# Read the merged text file into a DataFrame
df = pd.read_csv(merged_txt_path, delimiter='\t', names=['review'])

# Print the DataFrame
df.head(5)

,review
0,"特别赞的一家商场,在王府井这个商场云集和大牌云集的地方,apm商场算是人气特别高的了,交通,..."
1,"LINLEE在王府井apm也开新店了,超级喜欢他们家的口味,和朋友逛街无意看到的,果断去买一..."
2,"北京apm,东城区商场热门榜第一名️,王府井大街号,环境,商场环境干净整齐,布局分明,美食购..."
3,"一定要错过下午点半以后,不然吃饭只能排队等半个小时,而且拿到号了一定别因为还有十几桌就走开,..."
4,"花,Young的年华,年️日,月日,北京apm首层中庭,迎来位艺术家作品联展,春意盎然的️,..."


## 4. Convert to csv file for storage

In [20]:
# Define the list of txt file paths that need to be converted
txt_paths = ['../data/review/reviews_1.txt', '../data/review/reviews_2.txt', '../data/review/reviews_3.txt', '../data/review/reviews_4.txt', '../data/review/reviews_merged.txt']

# Loop through each txt file and convert it to a csv file
for txt_path in txt_paths:
    # Generate the corresponding csv file path
    csv_path = txt_path.replace('.txt', '.csv')
    
    # Open the txt file and csv file
    with open(txt_path, 'r', encoding='utf-8') as f_txt, open(csv_path, 'w', encoding='utf-8', newline='') as f_csv:
        # Create a csv.writer object to write data to the csv file
        writer = csv.writer(f_csv)
        
        # Write the header row
        writer.writerow(['review'])
        
        # Read each line in the txt file and write it to the csv file
        for line in f_txt:
            # Wrap each line of data in a list to represent a row of data in the csv file
            writer.writerow([line.strip()])



## 5. Splitting

### Custom stopwords dictionary（two different versions）

In [21]:
# Define a list of file paths for the stop word files
stopword_paths = ['stopwords/source/hit_stopwords.txt', 'stopwords/source/scu_stopwords.txt', 'stopwords/source/baidu_stopwords.txt', 'stopwords/source/cn_stopwords.txt']

# Read all stop words into a single list
stopwords = []
for path in stopword_paths:
    with open(path, 'r', encoding='utf-8') as f:
        # Read each line in the file, strip whitespace, and add the resulting words to the list
        words = [w.strip() for w in f.readlines()]
        stopwords.extend(words)

# Remove duplicate stop words
stopwords = list(set(stopwords))

# Write all stop words to a file
with open('stopwords/stopwords.txt', 'w', encoding='utf-8') as f:
    # Join the stop words into a single string separated by newlines, and write to the file
    f.write('\n'.join(stopwords))

# Read adverb words from file
adv_path = 'sentiment_dic/adv_dic.txt'
with open(adv_path, 'r', encoding='utf-8') as f:
    adverbs = [w.strip() for w in f.readlines()]

# Remove adverb words from stopwords
stopwords = [w for w in stopwords if w not in adverbs]

# Read negative words from file
neg_path = 'sentiment_dic/negative_dic.txt'
with open(neg_path, 'r', encoding='utf-8') as f:
    negatives = [w.strip() for w in f.readlines()]

# Remove negative words from stopwords
stopwords = [w for w in stopwords if w not in negatives]

# Sort the stop words alphabetically
stopwords.sort()

# Write modified stop words to a file
with open('stopwords/stopwords_pro.txt', 'w', encoding='utf-8') as f:
    # Join the stop words into a single string separated by newlines, and write to the file
    f.write('\n'.join(stopwords))


### Split word processing

In [22]:
# Define the file paths to read from and write to
input_file_paths = ['../data/review/reviews_1.txt', '../data/review/reviews_2.txt', '../data/review/reviews_3.txt', '../data/review/reviews_4.txt']
output_directory = '../data/review/processed/'

# Set the file path for the stop words
stopword_path = 'stopwords/stopwords_pro.txt'

# Create a set of stop words by opening the file and using a set comprehension
with open(stopword_path, 'r', encoding='utf-8') as f:
    stopwords = {word.strip() for word in f}

# Load the custom dictionary
custom_dict_path = 'stopwords/custom_dict.txt'
jieba.load_userdict(custom_dict_path)

# Define a function to process each review text
def process_text(text):
    # Tokenize the text with Jieba and enable HMM
    tokens = jieba.cut(text, cut_all=False, HMM=True)
    # Remove stop words and punctuation
    tokens = [token for token in tokens if token not in stopwords and token.strip() and not token.isnumeric()]
    # Return the processed text as a string
    return ' '.join(tokens)

# Loop over each file, process the review text and save it to a new text file
for input_file_path in input_file_paths:
    # Create the output file path
    output_file_path = os.path.join(output_directory, os.path.basename(input_file_path))
    # Open the input and output files
    with open(input_file_path, 'r', encoding='utf-8') as input_file, open(output_file_path, 'w', encoding='utf-8') as output_file:
        # Process each line of the input file
        for line in input_file:
            # Process the review text
            processed_text = process_text(line.strip())
            # Write the processed text to the output file
            output_file.write(processed_text + '\n')